# Phase 3 Classification Project - NYC Cycling Fatalities 

## Business Problem and Overview

   Our proposed client is the City of New York itself. The city government has come under fire from constituents and cyclist advocacy groups for the dangerous conditions cyclists endure. The city's transportation leadership want to know what conditions lead to deadly traffic collisions for New York's cyclists. The classification model in this instance is not the end product, as once the city has a record of the accident, they already know if a cyclist has died or not. The final product of this analysis will be the emergent patters revealed by what parameters make for a classification model capable of predicting fatal cycling accidents.
   
   Fittingly, the City of New York is also the source of our data. Our dataset comes in the form of 1.8 million rows detailing motor vehicle collisions that have occured in NYC. The city publishes this data via the NYC Open Data project, an initiative undertaken by the city to make publicly aggregated data accessible to citizens. The data spans nearly a decade, covering dates from July 2012, and is updated almost daily. 

## 1. Data Understanding

### 1.1 Import the Relevant Libraries, Modules, and Functions

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# This pandas option makes sure the juptyer notebook displays all the columns 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 1.2 CSV Into Pandas Dataframe

In [3]:
# Here we read our csv into a pandas dataframe. This can take a while. 
df = pd.read_csv('../data/Motor_Vehicle_Collisions.csv', low_memory=False)

In [4]:
# df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830092 entries, 0 to 1830091
Data columns (total 29 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   CRASH DATE                     object 
 1   CRASH TIME                     object 
 2   BOROUGH                        object 
 3   ZIP CODE                       object 
 4   LATITUDE                       float64
 5   LONGITUDE                      float64
 6   LOCATION                       object 
 7   ON STREET NAME                 object 
 8   CROSS STREET NAME              object 
 9   OFF STREET NAME                object 
 10  NUMBER OF PERSONS INJURED      float64
 11  NUMBER OF PERSONS KILLED       float64
 12  NUMBER OF PEDESTRIANS INJURED  int64  
 13  NUMBER OF PEDESTRIANS KILLED   int64  
 14  NUMBER OF CYCLIST INJURED      int64  
 15  NUMBER OF CYCLIST KILLED       int64  
 16  NUMBER OF MOTORIST INJURED     int64  
 17  NUMBER OF MOTORIST KILLED      int64  
 18  CO

### 1.3 Seperating by Cyclist Fatalities, Cyclist Injuries and Finding Distributions

In [4]:
# Lets start by finding how many collisions involved a cyclist, fatally or otherwise. 
cyclist_collisions = df.loc[ (df['NUMBER OF CYCLIST INJURED'] > 0) | (df['NUMBER OF CYCLIST KILLED'] > 0)]
len(cyclist_collisions)

43280

In [5]:
# This df is comprised of collisions where one or more cyclist were injured, but none died. 
non_lethal_collisions = df.loc[(df['NUMBER OF CYCLIST INJURED'] > 0) & (df['NUMBER OF CYCLIST KILLED'] == 0)]
len(non_lethal_collisions)

43101

In [6]:
# Next lets find how many rows record a cyclist dying. 
lethal_collisions = df.loc[df['NUMBER OF CYCLIST KILLED'] > 0]
len(lethal_collisions)

179

In [7]:
# Here we are determining how many collisions resulted in both cyclist injuries, and cyclist fatalities. 
combination_collisions = df.loc[ (df['NUMBER OF CYCLIST INJURED'] > 0) & (df['NUMBER OF CYCLIST KILLED'] > 0) ]
len(combination_collisions)

7

In [8]:
# Only one row details an event where more than one cyclist died.
print(len(df.loc[df['NUMBER OF CYCLIST KILLED'] > 1]))

# This row describes the tragic terrorist attack that occured on Halloween 2017.
df.loc[df['NUMBER OF CYCLIST KILLED'] == 2]

1


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
687327,10/31/2017,15:08,MANHATTAN,10014,40.729046,-74.01073,"(40.729046, -74.01073)",WEST STREET,WEST HOUSTON STREET,NaN,12.0,8.0,7,6,1,2,4,0,Other Vehicular,Unspecified,Unspecified,Unspecified,Unspecified,3782508,Flat Bed,Bus,Bike,Bike,Bike


### 1.4 Counts Analysis 
    
   So out of nearly 2 million recorded traffic collisions, 43,280 involved a cyclist either getting injured, or killed. 
   
   Of those 43,280 collisions involving a cyclist, 43,101 detail a cyclist getting injured, but no cyclists dying. This means that in 99.5% of traffic collisions where a cyclist is either injured or killed, the cyclist does not die of their injuries. This is a HIGHLY imbalanced dataset, and our classifiers will need to be extremely robust to detect the minority class. 
   
   We found 179 rows detailing an incident that resulted in the death of a cyclist. Only 7 rows fell into the category of 'combination collisions' where there were both cyclist injuries, as well as cyclist deaths. These combination rows will be considered lethal collisions in our classification, as they have obvious crossed the threshold for what defines a deadly collision (more than 0 cyclist deaths). 
   
   Only one row in our entire dataset represents an event where more than one cyclist died. This row details the terrorist attack that occured on Halloween of 2017, where an attacker drove a pickup truck down the westside bike path, injuring 12, and killing 8. This tragic event is certainly an outlier in our dataset, and will not be included in our model building process, as anti-terrorist measures on seperated, dedicated bike paths is outside the scope of this project's analysis. In addition, measures to prevent this kind of event from happening again on the west side bike path have already been taken. 

In [9]:
# Isolating just the collisions involving cyclists
cyc_col = df.loc[ (df['NUMBER OF CYCLIST INJURED'] > 0) | (df['NUMBER OF CYCLIST KILLED'] > 0)]

# Dropping the 2017 terror attack from our dataframe
all_cycling_collisions = cyc_col.loc[cyc_col['NUMBER OF CYCLIST KILLED'] < 2]

# Making sure we don't have any collisions recorded twice, by dropping any duplicates in the ID column 
all_cycling_collisions.drop_duplicates(['COLLISION_ID'])

# Checking the length 
len(all_cycling_collisions)

43279

In [10]:
# We saved our cycling collisions data as its own csv, as this is really the raw dataset of interest. 

#all_cycling_collisions.to_csv('Cycling_Collisions_Isolated')

### 1.5 NaN Value Report Function 

In [11]:
# This custom function lets us check the number of NaN values for each of the columns specified in the list
def nan_report(columns):

    '''This function takes in a list of column names, and will output the count and percents of NaN values
       in each of those columns in the lethal and non-lethal dataframes.'''
    
    for col in columns:
        
        # First we will get the raw counts for the lethal collisions, then convert to %'s
        lethal_nans = (lethal_collisions[col].isna().sum())
        lethal_percent = int(round((lethal_nans / 178) * 100))
        
        # Then do the same for the non_lethal_collisions
        non_lethal_nans = (non_lethal_collisions[col].isna().sum())
        non_lethal_percent = int(round((non_lethal_nans / 43101) * 100))
        
        # Printing the results in a highly readable way
        print(f'''Column: {col.title()}
        
        Lethal Collisions:     {lethal_percent}% NaN values.
        Non-Lethal Collisions: {non_lethal_percent}% NaN values.
        
        
        ''')

### 1.6 Column Data and NaN Value Counts

   The first problem with much of the information in this dataset, is that it is stored in messy, irregular strings. For example under vehicle type, we see 'taxi' and 'Taxi' as seperate categories. In the date column, the dates are stored as strings, which will have to be converted, or at the very least one hot encoded into buckets if we want date to be a useful feature. 
   
   The other challenge standing in the way of this analysis is going to be dealing with all the NaN values in columns that we need for our models. With this is mind, there are two reasons why we chose to not simply drop rows with NaN values in important input columns. 

   One is that because our dataset is so incredibly imbalanced, NaN values in important columns, in rows belonging to our minority class, will significantly impede our attempt to generate a generalizable model that can identify the minority class in new unseen data. The other reason we must find a way to deal with the rows containing NaNs is that since the number of lethal collisions is so proportionally small, if we just dropped rows with NaNs in the relevant columns, we would be thinning out our already tiny minority class. 
   
   This means that our process is going to involve a lot of preprocessing, filling in the missing pieces. The first step in that process is to see what the damage is, and look at the NaN value counts for our columns.

### 1.6.1 Time and Location Data Columns


In [12]:
# Displaying the time and location columns 
all_cycling_collisions.iloc[:,:10].head(5)

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME
52,04/16/2021,11:00,QUEENS,11368,40.749580,-73.865410,"(40.74958, -73.86541)",NaN,NaN,100-10 ROOSEVELT AVENUE
90,04/14/2021,0:00,NaN,NaN,40.601864,-74.002320,"(40.601864, -74.00232)",BATH AVENUE,NaN,NaN
139,04/13/2021,17:55,BRONX,10452,40.844105,-73.923065,"(40.844105, -73.923065)",GRANT HIGHWAY,UNIVERSITY AVENUE,NaN
145,04/14/2021,19:45,BROOKLYN,11201,40.694840,-73.983910,"(40.69484, -73.98391)",FLATBUSH AVENUE EXTENSION,JOHNSON STREET,NaN
178,04/16/2021,0:30,QUEENS,11369,40.758680,-73.875520,"(40.75868, -73.87552)",93 STREET,32 AVENUE,NaN


In [13]:
# Making a list of the time and location columns, then passing them into the nan_report function
time_and_location_columns = ['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
                             'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
                             'OFF STREET NAME']

nan_report(time_and_location_columns)

Column: Crash Date
        
        Lethal Collisions:     0% NaN values.
        Non-Lethal Collisions: 0% NaN values.
        
        
        
Column: Crash Time
        
        Lethal Collisions:     0% NaN values.
        Non-Lethal Collisions: 0% NaN values.
        
        
        
Column: Borough
        
        Lethal Collisions:     27% NaN values.
        Non-Lethal Collisions: 22% NaN values.
        
        
        
Column: Zip Code
        
        Lethal Collisions:     27% NaN values.
        Non-Lethal Collisions: 22% NaN values.
        
        
        
Column: Latitude
        
        Lethal Collisions:     13% NaN values.
        Non-Lethal Collisions: 7% NaN values.
        
        
        
Column: Longitude
        
        Lethal Collisions:     13% NaN values.
        Non-Lethal Collisions: 7% NaN values.
        
        
        
Column: Location
        
        Lethal Collisions:     13% NaN values.
        Non-Lethal Collisions: 7% NaN values.
 

### 1.6.2 Contributing Factor Columns

In [14]:
# Lets examine the contributing factor columns 
all_cycling_collisions.iloc[:,18:23].head(5)

,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5
52,Turning Improperly,Unspecified,NaN,NaN,NaN
90,Failure to Yield Right-of-Way,Unspecified,NaN,NaN,NaN
139,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,Unspecified,NaN,NaN,NaN
145,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN
178,Unsafe Speed,Unspecified,Unspecified,NaN,NaN


In [15]:
# Passing a list of the columns into our nan_report function
contributing_factor_columns = ['CONTRIBUTING FACTOR VEHICLE 1',
                               'CONTRIBUTING FACTOR VEHICLE 2',
                               'CONTRIBUTING FACTOR VEHICLE 3',
                               'CONTRIBUTING FACTOR VEHICLE 4',
                               'CONTRIBUTING FACTOR VEHICLE 5']

nan_report(contributing_factor_columns)

Column: Contributing Factor Vehicle 1
        
        Lethal Collisions:     0% NaN values.
        Non-Lethal Collisions: 0% NaN values.
        
        
        
Column: Contributing Factor Vehicle 2
        
        Lethal Collisions:     7% NaN values.
        Non-Lethal Collisions: 7% NaN values.
        
        
        
Column: Contributing Factor Vehicle 3
        
        Lethal Collisions:     87% NaN values.
        Non-Lethal Collisions: 98% NaN values.
        
        
        
Column: Contributing Factor Vehicle 4
        
        Lethal Collisions:     97% NaN values.
        Non-Lethal Collisions: 100% NaN values.
        
        
        
Column: Contributing Factor Vehicle 5
        
        Lethal Collisions:     98% NaN values.
        Non-Lethal Collisions: 100% NaN values.
        
        
        


### 1.6.3 Vehicle Type Code Columns

   These 5 columns contain data about the vehicles that were involved in the collision, including the Bike/E-Bike. Vehicle type could be a super important feature for our models, so we will have to deal with the messy string values, as well as NaN values. 

In [16]:
# Displaying the vehicle type code columns
all_cycling_collisions.iloc[:,24:29].head(5)

,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
52,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN
90,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN
139,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN
145,Sedan,Bike,NaN,NaN,NaN
178,Sedan,Bike,NaN,NaN,NaN


So just looking at these first ten rows, it becomes obvious that the majority of collisions are between a cyclist and one other vehicle, to confirm this, lets take a look at how the number of NaN values increase over the course of the five columns.

In [17]:
# Passing the type code columns into our nan_report function
vehicle_type_columns = ['VEHICLE TYPE CODE 1',
                        'VEHICLE TYPE CODE 2',
                        'VEHICLE TYPE CODE 3',
                        'VEHICLE TYPE CODE 4',
                        'VEHICLE TYPE CODE 5']

nan_report(vehicle_type_columns)

Column: Vehicle Type Code 1
        
        Lethal Collisions:     0% NaN values.
        Non-Lethal Collisions: 0% NaN values.
        
        
        
Column: Vehicle Type Code 2
        
        Lethal Collisions:     7% NaN values.
        Non-Lethal Collisions: 9% NaN values.
        
        
        
Column: Vehicle Type Code 3
        
        Lethal Collisions:     87% NaN values.
        Non-Lethal Collisions: 98% NaN values.
        
        
        
Column: Vehicle Type Code 4
        
        Lethal Collisions:     97% NaN values.
        Non-Lethal Collisions: 100% NaN values.
        
        
        
Column: Vehicle Type Code 5
        
        Lethal Collisions:     98% NaN values.
        Non-Lethal Collisions: 100% NaN values.
        
        
        


We can see that only 13% and 2% of the rows in our lethal, and non-lethal dataframes respectively, have values for Vehicle Type by the time we get to the third column. The string values, and the NaN values will be dealt with in the data preparation step.

## 2. Data Preparation

The first step of preprocessing is to drop the columns that are not useful for this analysis.

In [18]:
all_cycling_collisions = all_cycling_collisions.drop([ 'NUMBER OF PERSONS INJURED',
                                                       'NUMBER OF PERSONS KILLED', 
                                                       'NUMBER OF PEDESTRIANS INJURED',
                                                       'NUMBER OF PEDESTRIANS KILLED',  
                                                       'NUMBER OF MOTORIST INJURED',
                                                       'NUMBER OF MOTORIST KILLED',
                                                       'COLLISION_ID' ] , axis=1)

all_cycling_collisions.shape



(43279, 22)

Before we can seperate our training data and our holdout data, we need a target y column (lethality), as right now, our dataset stores this information in two columns. After we have created this new column, we can drop the original two. 

In [19]:
all_cycling_collisions['Lethal'] = 0
all_cycling_collisions.loc[ all_cycling_collisions['NUMBER OF CYCLIST KILLED'] > 0],['Lethal'] = 1

SyntaxError: cannot assign to literal (<ipython-input-19-902c7bdc508a>, line 2)

The next step is to seperate our data into the training, and holdout sets to avoid any possible data leakage. 


In [ ]:


X_train, X_hold, y_train, y_hold = train_test_split(all_cycling_collisions, random_state=6, test_size=.15)

### 2.1 Dates

   Dates are a feature we certainly want to include in our model, but as the column stands right now, 